In [ ]:
#from docopt import docopt
#import anndata as ad
#import scanpy as sc
#import pandas as pd
#import scanpy_helpers as sh
#import decoupler as dc
#from tqdm import tqdm
#import re 
#import matplotlib.pyplot as plt
#import seaborn as sns
#from math import ceil
#from itertools import zip_longest

In [ ]:
import seaborn as sns
import pandas as pd

In [ ]:
import scanpy as sc

In [ ]:
import scirpy as ir

In [ ]:
path_tcr_input = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/scrna_seq_data/vdj_t/10mix1_filtered_contig_annotations.csv"

In [ ]:
df_tcr_raw = pd.read_csv(path_tcr_input)

In [ ]:
df_tcr_raw

In [ ]:
df_tcr_raw["productive"] = df_tcr_raw["productive"].astype(str)
print(f"Total measurements: {len(df_tcr_raw)}")
df_tcr_raw.head(5)

In [ ]:
df_tcr_raw.columns

In [ ]:
df_tcr_raw.full_length

In [ ]:
columns = [
    "barcode",
    "v_gene",
    "d_gene",
    "c_gene",
    "j_gene",
    "productive",
    "full_length",
]
df_tcr_raw[~df_tcr_raw["full_length"]][columns].head()

In [ ]:
columns += ["cdr3", "cdr3_nt"]

In [ ]:
df_tcr_raw[(df_tcr_raw["productive"] == "False") & (df_tcr_raw["full_length"])][columns].head(5)

In [ ]:
# The column 'productive' contains mixed data types which are not compatible with downstream tools.
# We correct them by casting them to strings.

In [ ]:
df_tcr_raw["productive"] = df_tcr_raw["productive"].astype(str)

In [ ]:
print(f"Total measurements: {len(df_tcr_raw)}")

In [ ]:
df_tcr_raw.head(5)

In [ ]:
path_tcr_csv = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/merged/2021-02-01_sorted_cd8_til_merged_data.h5ad"

In [ ]:
adata_tcr = sc.read_h5ad(path_tcr_csv)

In [ ]:
print(f"Amount cells: {len(adata_tcr)}")

In [ ]:
adata_tcr.var.tail(5)

In [ ]:
# Create a boolean mask for genes that do not start with "ENS"
gene_mask = ~adata_tcr.var_names.str.startswith('ENS')

# Filter the AnnData object based on the mask
filtered_adata = adata_tcr[:, gene_mask].copy()

In [ ]:
adata_tcr

In [ ]:
path = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/scrna_seq_data/adata_10mix-ICI1.h5ad"

adata_10mix = sc.read_h5ad(path)

In [ ]:
adata_10mix

In [ ]:
import muon as mu

In [ ]:
path_contig_annotation= "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/scrna_seq_data/vdj_t/11mix1_filtered_contig_annotations.csv"

In [ ]:
matrix_path = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/11mix-ICI1/outs/per_sample_outs/11mix-ICI1/count/sample_filtered_feature_bc_matrix.h5"

In [ ]:
# Load the TCR data
adata_tcr = ir.io.read_10x_vdj(
    path_contig_annotation
)

# Load the associated transcriptomics data
adata = sc.read_10x_h5(
    matrix_path
)
adata.var_names_make_unique()

In [ ]:
mdata = mu.MuData({"gex": adata, "airr": adata_tcr})

In [ ]:
mdata

In [ ]:
sc.pp.log1p(mdata["gex"])
sc.pp.pca(mdata["gex"], svd_solver="arpack")
sc.pp.neighbors(mdata["gex"])
sc.tl.umap(mdata["gex"])

In [ ]:
ir.pp.index_chains(mdata)
ir.tl.chain_qc(mdata)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 4), gridspec_kw={"wspace": 0.5})
#mu.pl.embedding(mdata, basis="gex:umap", color=["CD3E"], ax=ax0, show=False)
mu.pl.embedding(mdata, basis="gex:umap", color=["airr:receptor_type"], ax=ax1)